In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

app = Flask(__name__)

# Загрузка данных и обучение модели ARIMA
df = pd.read_csv('your_data.csv')  # Замените 'your_data.csv' на путь к вашим данным
model = ARIMA(df['total_load'], order=(5, 1, 2))
model_fit = model.fit()

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()
    forecast_steps = data['forecast_steps']
    forecast = model_fit.forecast(steps=forecast_steps)
    return jsonify({'forecast_values': forecast.tolist()})

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import streamlit as st
import requests

# Функция для отправки POST запроса к Flask API
def get_forecast(forecast_steps):
    url = 'http://127.0.0.1:5000/predict'  # Замените на URL вашего Flask API
    data = {'forecast_steps': forecast_steps}
    response = requests.post(url, json=data)
    return response.json()['forecast_values']

# Заголовок и описание приложения
st.title('Прогноз загруженности станций')
st.write('Введите количество дней для прогноза')

# Пользовательский ввод
forecast_steps = st.number_input('Количество дней', min_value=1, max_value=365*2, value=365)

# Получение прогноза от Flask API
forecast_values = get_forecast(forecast_steps)

# Визуализация прогноза
st.line_chart(forecast_values)
